In [64]:
import pandas as pd
import numpy as np
import ast

In [65]:
df_film = pd.read_csv("df_film.csv")

In [3]:
Top_pays = pd.read_csv("Top_pays.csv")

In [4]:
top10_real_note = pd.read_csv('top10_real_note.csv')

## Ajout données de tmdb

Pour ajouter des données telles que la popularité, les pays de production, les affiches de films, nous avons du nous servir du tmdb.csv

In [5]:
df_tmbd = pd.read_csv("tmdb_full.csv")

/var/folders/fk/s9b7316s21b6krxxntv4pzzm0000gn/T/ipykernel_2421/4031644567.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd = pd.read_csv("tmdb_full.csv")


In [ ]:
df_tmbd = df_tmbd.drop(columns = [
    "adult",
    "backdrop_path",
    "budget",
    "genres",
    "homepage",
    "id",
    "original_language",
    "original_title",
    "overview",
    "release_date",
    "revenue",
    "runtime",
    "spoken_languages",
    "tagline",
    "title",
    "video",
    "vote_average",
    "vote_count",
    "production_companies_name",
    "production_companies_country"])

Merge nos nouvelles données avec le df_film 

In [ ]:
df_film = pd.read_csv("df_film.csv")
df_merge = df_film.merge(df_tmbd, left_on = "id_film", right_on = "imdb_id", how = "left")
df_merge = df_merge.drop(columns = ["Unnamed: 0.2","Unnamed: 0.1", "Unnamed: 0", "tconst"])

## Créer des colonnes supplémentaires

Créer la colonne décénie

In [9]:
df_film["decenie"] = ((df_film['année'] // 10) * 10).astype(str)

## Remplacer des valeurs manquantes

Transformer les valeurs nulles en format compréhensible par python. Dans notre df actuellement les valeurs nulles sont des strings '\\N'

In [10]:
df_film = df_film.replace('\\N', np.nan)

### Durée (temps_minutes)

Ici, on veut remplacer les valeurs nulles par la moyenne des autres films de la même décénie

Converti le temps (str) en int en contournant les valeurs nulles (fonctionne uniquement avec un type compris par python)

In [ ]:
df_film['temps_minutes'] = df_film['temps_minutes'].astype('Int64')

Cacul de la moyenne par décénie

In [12]:
moy_decenie = df_film.groupby("decenie")["temps_minutes"].mean()

fonction qui remplace les valeurs nulles par la moyenne de la décénie à laquelle il appartient

In [13]:
def replacena_mean(row):
    if str(row["temps_minutes"]) == "<NA>":
        row["temps_minutes"] = moy_decenie.loc[row["decenie"]]
    return row

In [14]:
# Appliquer cette fonction
df_film = df_film.apply(replacena_mean, axis=1)

In [15]:
df_film.to_csv("df_film.csv")

### Années

In [16]:
# Création d'une liste de films qui ont pour année 0
films_annee0 = df_film.loc[df_film["année"] == 0, "titre"].tolist()

In [17]:
# Dictionnaire avec les années manquantes au DF
films = {
    "Shubh Kaamna": 1983,
    "You Don't Know Nicotine": 2020,
    "Becoming Led Zeppelin": 2021,
    "Concorde, le rêve supersonique": 2020,
    "Luccas Neto em: Acampamento de Férias": 2019,
    "Luccas Neto em: Acampamento de Férias 2": 2020,
    "Prem Prakaran": 2021,
    "El Arribo de Conrado Sierra": 2012,
    "Cyborg Nemesis: The Dark Rift" : 2014,
    "90° South" : 1933,
    "Samhain" : 2021,
    "Haunted Connecticut" : 2009,
}

# Apply du dictionnaire pour remplacer les valeurs manquantes
df_film["année"] = df_film.apply(lambda row: films.get(row["titre"], row["année"]), axis=1)

In [18]:
# Supression des lignes restantes avec année non renseignée
df_film= df_film[df_film["année"] != 0]

# Analyse

## Genre les + vus par décénie

La colonne genre contient une string avec plusieurs genres différents. Je veux dans un premier temps transformer ces chaînes de caractères en listes.

Pour éviter les bugs avec split, je remplace les valeurs manquantes par des liste vides.

In [19]:
df_film['genre'] = df_film['genre'].fillna("")

Je créé donc une fonction qui va prendre la chaîne de caractère en paramètre et renvoyer la liste pour chaque ligne

In [20]:
def transfo_liste(chaine:str)-> list:
    liste_genres = []
    for genre in chaine.split(","):
        liste_genres.append(genre.strip())
    return liste_genres


Je l'applique sur la colonne que je veux. Ici, je transforme la colonne genre (initialement des strings) en une nouvelle colonne genre qui contient maintenant des listes de genre.

In [21]:
df_film['genre'] = df_film['genre'].apply(transfo_liste)

Maintenant, je voudrais compter les nombres de films de chaque genres par décénie. Pour ça je veux faire un dictionnaire qui à en clé la décénie, et en valeur, les nombres de films par genre [pour cette même décénie donc].

In [22]:
resultats_par_decennie = {}

for decenie in df_film["decenie"].unique():
    df_dec = df_film.loc[df_film["decenie"] == decenie, ['genre', 'decenie']].explode("genre")
    nb_genres = df_dec.groupby('genre')["genre"].count()
    resultats_par_decennie[decenie] = nb_genres

Je voudrais transformer ce dictionnaire en dataframe pour ensuite faire des visualisation graphiques

In [23]:
df_genre_decenie = pd.DataFrame(resultats_par_decennie).fillna(0)

### Objectif: heatmap

L'objectif est de réaliser une heatmap (dans la partie analyse). Une heatmap qui prend en axe horizontal la décénie et en axe vertical les 10 genres les plus populaires. La couleur ne doit pas représenter le nombre de films brut mais plutôt le ratio que chaque genre représente par décénie. 

1) On commence à travailler sur tout les genres (pas le top 10 pour l'instant)

Pour calculer le ration il faut faire = case / somme_colonne, on prendra les cases du dataframe top 10 qu'on divisera par la somme par colonne ce df plus global. On calcul donc la somme par colonne (par décénie)

In [24]:
sum_par_dec = df_genre_decenie.sum(axis = 0) #calcul de la somme_colonne

2. Pour le top 10, je fais une somme des genres que je trie par ordre croissant et desquels je ne garde que le top 10 (head10)

In [25]:
df_genre_decenie["sum_genre"] = df_genre_decenie.sum(axis = 1)
df_top_genre = df_genre_decenie.sort_values("sum_genre", ascending = False).head(10)

Je supprimme ensuite la colonne sum_genre qui ne servait qu'à créer ce nouveau dataframe

In [26]:
df_top_genre = df_top_genre.drop(columns = 'sum_genre')

Maintenant je fais le ratio avec la somme par colonne du dataframe total (sans le filtre du top10)

In [27]:
df_top_genre = df_top_genre / sum_par_dec

In [28]:
df_top_genre.to_csv("df_top_genre.csv")

## Meilleur score pays producteur

In [29]:
# Convertir liste pays
def convertir_en_liste(x):
    if isinstance(x, str):  # Si c'est une chaîne
        return ast.literal_eval(x)  # Convertir en liste Python
    return x  # Retourner inchangé si ce n'est pas une chaîne

df_film["production_countries"] = df_film["production_countries"].apply(convertir_en_liste)

In [30]:
# explode en DF Pays / nb_film
df_exploded = df_film.explode("production_countries")


In [31]:
nb_films_pays = df_exploded["production_countries"].value_counts().reset_index()

In [32]:
nb_films_pays.head()

,index,production_countries
0,US,46015
1,FR,11799
2,IN,9998
3,GB,9708
4,JP,8526


In [33]:
# Rename colonne
nb_films_pays = nb_films_pays.rename(columns={"production_countries" :"pays", "count": "nb_films"})

In [34]:
# Explode en DF pays/popu
pop_pays = df_exploded.groupby("production_countries")["popularity"].mean().sort_values(ascending=False).reset_index()

In [35]:
# Rename colonne
pop_pays = pop_pays.rename(columns={"production_countries" :"pays"})

In [ ]:
# Merge nb_films_pays et pop_pays
df_score = nb_films_pays.merge(pop_pays, right_on = pop_pays['pays'], left_on = nb_films_pays['pays'])

In [ ]:
df_score = df_score.drop(columns = ["key_0","pays_x"]).rename(columns = {"pays_y":"pays"})

In [ ]:
# Moyenne de film par pays
moy_nb_film = df_score["nb_films"].mean()

In [40]:
# Fonction pour calculer le score
def score(row):
    return 0.2 + (min(moy_nb_film, row["nb_films"]) / moy_nb_film) * 0.8

In [41]:
# Apply de la fonction dans la nouvelle colonne ratio
df_score["ratio"] = df_score.apply(score,axis=1)

In [42]:
# Nouvel colonne score avec le calcul ratio
df_score["score"] = df_score["popularity"] * df_score["ratio"]

In [ ]:
# Top 15 pays
df_score = df_score.sort_values("score", ascending=False).head(15)

In [44]:
# Dictionnaire de correspondance code -> pays
pays = {
    "US": "États-Unis",
    "GB": "Royaume-Uni",
    "IE": "Irlande",
    "KR": "Corée du Sud",
    "CA": "Canada",
    "AU": "Australie",
    "BE": "Belgique",
    "JP": "Japon",
    "CN": "Chine",
    "NZ": "Nouvelle-Zélande",
    "ES": "Espagne",
    "FR": "France",
    "NO": "Norvège",
    "HK": "Hong Kong",
    "NL": "Pays-Bas",
    "DE": "Allemagne",
    "CZ": "République tchèque",
    "IT": "Italie",
    "MX": "Mexique",
    "TH": "Thaïlande"
}
# Apply du nom du pays au lieu du code
df_score["pays"] = df_score["pays"].map(pays)

In [45]:
top_pays = df_score.sort_values("score", ascending=False)

In [46]:
#top_pays.to_csv("Top_pays.csv")

## Meilleure score réalisateur

In [16]:
#Group by + agg
top_real = (
    df_film.groupby("Real")
    .agg({"titre": "count", "nb_votes": "sum", "note": "mean"})
    .sort_values(by="titre", ascending=False) 
    .reset_index())

In [17]:
# Rename colonne
top_real = top_real.rename(columns={"titre" :"nb_films"})

In [18]:
# Moyenne de nombre de votes
moy_vote = df_film["nb_votes"].mean()

In [19]:
# Moyenne de nombre de film
moy_nb_film_real = top_real["nb_films"].mean()

In [20]:
# Fonction pour calculer le score
def score2(row):
    return ((((min(moy_vote, row["nb_votes"]) / moy_vote) * 0.3) + ((min(moy_nb_film_real, row["nb_films"]))/ moy_nb_film_real) * 0.4) + 0.3)

In [21]:
# Nouvelle colonne avec colonne ratio
top_real["ratio"] = top_real.apply(score2,axis=1)

In [22]:
# Nouvelle colonne avec le score
top_real["score"] = top_real["note"] * top_real["ratio"]

In [23]:
# Top 10 real
top10_real = top_real.sort_values("score", ascending=False).head(10)

In [ ]:
#top10_real.to_csv("top10_real_note.csv")

In [24]:
# Fonction pour calculer le score
def real2(row):
    return ((((min(1000000, row["nb_votes"]) / 1000000) * 0.3) + ((min(10, row["nb_films"]))/ 10) * 0.4) + 0.3)

In [25]:
# Analyse avec d'autre filtres
top_real2 = top_real
top_real2["ratio"] = top_real2.apply(real2,axis=1)
top_real2["score"] = top_real2["note"] * top_real2["ratio"]

In [26]:
# Top 10
top10_real2 = top_real2.sort_values("score", ascending=False).head(10)

In [27]:
#Export CSV
top10_real2.to_csv("top10_real_note2.csv")

# Filtrer

### Retirer genre Adult

In [66]:
df_film = df_film[df_film['genre'].str.contains('Adult') == False]

## Par pays

On ne garde que les 20 pays avec les films les plus populaires & les films européens 

In [68]:
def contient_pays(value):
    pays_selectionnes = ['US', 'FR','ES','IT','DE','GB','PL','BE','NL','NO','CA','KR', 'IE','AU','JP','CN','NZ','HK','CZ','MX','TH']
    return any(pays in str(value) for pays in pays_selectionnes)

In [69]:
df_film_filtre = df_film[df_film["production_countries"].apply(contient_pays)]

>shape = 107 804 

In [ ]:
#df_film_filtre.to_csv("df_film_filtre.csv")

In [77]:
df_film_filtre = df_film_filtre[["titre", "année", "temps_minutes", "note", "nb_votes", "Real", "popularity", "poster_path", "production_countries", "genre"]]

In [79]:
#df_film_filtre.to_csv('df_film_filtre.csv')

## Création csv pour l'usage de gemini

In [3]:
df_gemini = pd.read_csv("gemini.csv")

In [7]:
df_gemini = df_gemini.drop(columns = ["id"])

In [15]:
df_gemini["année"] = df_gemini["année"].astype(str)

In [17]:
df_gemini["id"] = df_gemini['titre'] + " - " + df_gemini["année"]

In [ ]:
#df_gemini['id'].to_csv("gemini_id.csv", index = False)